# Identify Orphan Servers

When a data source is deleted in Purview, the corresponding assets are not deleted. This script is meant to identify servers that are not associated to any data source.

## Pre-requisites

To be able to use this notebook, you will need to have:
- Purview Account details
- A user authorized to access Purview APIs
- A Python environment with the required libraries

## Steps
- Update notebook with your environment information (purview account name, working directory & entities/server types to look for)
- Execute the cells in the notebook
- Load data in Excel and compare the list of servers to the list of datasources

In [1]:
# Import libraries

import os
import datetime
import pandas as pd

from io import BytesIO
from azure.identity import DefaultAzureCredential
from azure.purview.catalog import PurviewCatalogClient
from azure.purview.scanning import PurviewScanningClient

In [2]:
purview_account = '[ADD YOUR PURVIEW ACCOUNT NAME]'

working_directory = 'C:\\temp\\'

entitiesFilter = [
    'azure_sql_server',
    'azure_cosmosdb_account',
    'azure_storage_account'
]

In [3]:
def purview_client(purview_account):
    credential = DefaultAzureCredential()
    client = PurviewCatalogClient(
        endpoint=f'https://{purview_account}.purview.azure.com', 
        credential=credential,
        logging_enable=True)
    return client

In [4]:
def create_filter(entities):
    entityTypes = [{"entityType": entity} for entity in entities]
    filter = {"or": entityTypes}
    return filter

In [5]:
def create_search_body(keywords, filter):
    search_body = {
        'keywords': keywords if keywords else None,
        'facets': None,
        'filter': filter if filter else None,
    }
    return search_body

In [6]:
def query_to_dataframe(purview_client, filter, keywords ='*'):
    search_request = create_search_body(keywords, filter)
    purview_search = purview_client.discovery.query(search_request=search_request)
    search_df = pd.DataFrame.from_dict(purview_search['value'])

    output_df = pd.DataFrame({
        'name': search_df['name'],
        'entityType': search_df['entityType'],
        'createTime': pd.to_datetime(search_df['createTime'], unit='ms'),
        'updateTime': pd.to_datetime(search_df['updateTime'], unit='ms')
    })
    return output_df

In [7]:
def datasources_to_dataframe():
    credential = DefaultAzureCredential()
    client = PurviewScanningClient(
        endpoint=f'https://{purview_account}.scan.purview.azure.com', 
        credential=credential)
    response = client.data_sources.list_all()
    datasources = [datasource for datasource in response]

    def extract_datasource_property(properties, propertyName):
        if properties and propertyName in properties:
            return properties[propertyName]
        return None
            
    datasources_raw_df = pd.DataFrame(datasources)

    datasources_df = pd.DataFrame({
        'name': datasources_raw_df['name'],
        'kind': datasources_raw_df['kind'],
        'creationType': datasources_raw_df['creationType'],
        'endpointInfo': datasources_raw_df['properties'].apply(lambda x: 
                                                               extract_datasource_property(x, 'serverEndpoint') if extract_datasource_property(x, 'serverEndpoint') 
                                                               else (extract_datasource_property(x, 'endpoint') if extract_datasource_property(x, 'endpoint') 
                                                                     else extract_datasource_property(x, 'accountUri'))),
        'endpointPropertyName': datasources_raw_df['properties'].apply(lambda x: 
                                                               'serverEndpoint' if extract_datasource_property(x, 'serverEndpoint') 
                                                               else ('endpoint' if extract_datasource_property(x, 'endpoint') 
                                                                     else ('accountUri' if extract_datasource_property(x, 'accountUri') else None))),
        # 'serverEndpoint': datasources_raw_df['properties'].apply(lambda x: extract_datasource_property(x, 'serverEndpoint')),
        # 'endpoint': datasources_raw_df['properties'].apply(lambda x: extract_datasource_property(x, 'endpoint')),
        # 'accountUri': datasources_raw_df['properties'].apply(lambda x: extract_datasource_property(x, 'accountUri')),
        'resourceName': datasources_raw_df['properties'].apply(lambda x: extract_datasource_property(x, 'resourceName')),
        'subscription': datasources_raw_df['properties'].apply(lambda x: extract_datasource_property(x, 'subscription')),
        'resourceGroup': datasources_raw_df['properties'].apply(lambda x: extract_datasource_property(x, 'resourceGroup')),
        'createdAt': datasources_raw_df['properties'].apply(lambda x: extract_datasource_property(x, 'createdAt')),
        'lastModifiedAt': datasources_raw_df['properties'].apply(lambda x: extract_datasource_property(x, 'lastModifiedAt'))
    })


    return datasources_df

In [8]:
def export_to_csv(dataframe, file_path, filename):

    #create directory from download_path if it doesn't exist
    if not os.path.exists(file_path):
        os.makedirs(file_path)

    output_file  = f'{file_path}\{filename}'
    pd.DataFrame.to_csv(dataframe, path_or_buf=output_file, index=False)
    
    return output_file

In [ ]:
try:
    if 'purview_initiated' not in globals():
        purview_client = purview_client(purview_account)
        purview_initiated = True
    
    filter = create_filter(entitiesFilter)

    pv_sources_df = datasources_to_dataframe()
    pv_search_df = query_to_dataframe(purview_client, filter)

    fileTS = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    sources_file = f'pv_datasources_{fileTS}.csv'
    servers_file = f'pv_servers_{fileTS}.csv'
    export_to_csv(pv_sources_df, working_directory, sources_file)
    export_to_csv(pv_search_df, working_directory, servers_file)
    
    print(f'Files exported successfully at: {working_directory}')
except Exception as e:
    print(e)

